In [1]:
import os 
os.environ["PATH"] += ':/Users/apple/Documents/setups/google-cloud-sdk/bin'

In [2]:
!which gsutil

/Users/apple/Documents/setups/google-cloud-sdk/bin/gsutil


# Make a the bucket in GCS

In [3]:
!gsutil mb gs://de_retail_db ## ! this is shell command

Creating gs://de_retail_db/...
ServiceException: 409 A Cloud Storage bucket named 'de_retail_db' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


In [4]:
!gsutil ls

gs://de_retail_db/


In [10]:
!gsutil cp -r ../data/retail_db gs://de_retail_db/retail_db

Copying file://../data/retail_db/create_db_tables_pg.sql [Content-Type=application/x-sql]...
Copying file://../data/retail_db/schemas.json [Content-Type=application/json]...
Copying file://../data/retail_db/load_db_tables_pg.sql [Content-Type=application/x-sql]...
Copying file://../data/retail_db/customers/part-00000 [Content-Type=application/octet-stream]...
- [4 files][ 10.7 MiB/ 10.7 MiB]  424.7 KiB/s                                   
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file://../data/retail_db/products/part-00000 [Content-Type=application/octet-stream]...
Copying file://../data/retail_db/departments/part-00000 [Content-Type=application/octet-stream]...
Copying file://../data/retail_db/order_items/part-00000 [Content-Type=application/octet-stream]...
Copying fil

In [13]:
!gsutil ls -r gs://de_retail_db

gs://de_retail_db/retail_db/:
gs://de_retail_db/retail_db/create_db_tables_pg.sql
gs://de_retail_db/retail_db/load_db_tables_pg.sql
gs://de_retail_db/retail_db/schemas.json

gs://de_retail_db/retail_db/categories/:
gs://de_retail_db/retail_db/categories/part-00000

gs://de_retail_db/retail_db/customers/:
gs://de_retail_db/retail_db/customers/part-00000

gs://de_retail_db/retail_db/departments/:
gs://de_retail_db/retail_db/departments/part-00000

gs://de_retail_db/retail_db/order_items/:
gs://de_retail_db/retail_db/order_items/part-00000

gs://de_retail_db/retail_db/orders/:
gs://de_retail_db/retail_db/orders/part-00000

gs://de_retail_db/retail_db/products/:
gs://de_retail_db/retail_db/products/part-00000


In [15]:
from google.cloud import storage

In [17]:
gsclient = storage.Client() #DefaultCredentialsError 

### Setup Credentials
gcloud auth application-default login

In [ ]:
gsclient = storage.Client()
buckets = gsclient.list_buckets() # list of buckets iterator
for bucket in buckets:
    print(bucket.name)

de_retail_db


In [ ]:
blobs = gsclient.list_blobs('de_retail_db') # list of blobs iterator - parameter is bucket name
for blob in blobs:
    print(blob.name)

retail_db/categories/part-00000
retail_db/create_db_tables_pg.sql
retail_db/customers/part-00000
retail_db/departments/part-00000
retail_db/load_db_tables_pg.sql
retail_db/order_items/part-00000
retail_db/orders/part-00000
retail_db/products/part-00000
retail_db/schemas.json


In [27]:
bucket = gsclient.get_bucket('de_retail_db')
type(bucket) # <class 'google.cloud.storage.bucket.Bucket'>

google.cloud.storage.bucket.Bucket

In [34]:
blobs = gsclient.list_blobs('de_retail_db') # list of blobs iterator
type(blobs) # <class 'list'>

google.api_core.page_iterator.HTTPIterator

In [ ]:
bucket = gsclient.get_bucket('de_retail_db')
blob = bucket.blob('pythondemo/retail_db/orders/part-00000') #in memory

In [44]:
!gsutil ls gs://de_retail_db/pythondemo

CommandException: One or more URLs matched no objects.


In [49]:
help(blob.upload_from_filename) #help on blob upload filename

Help on method upload_from_filename in module google.cloud.storage.blob:

upload_from_filename(filename, content_type=None, client=None, predefined_acl=None, if_generation_match=None, if_generation_not_match=None, if_metageneration_match=None, if_metageneration_not_match=None, timeout=60, checksum='auto', retry=<google.api_core.retry.retry_unary.Retry object at 0x10ea6b2e0>) method of google.cloud.storage.blob.Blob instance
    Upload this blob's contents from the content of a named file.
    
    The content type of the upload will be determined in order
    of precedence:
    
    - The value passed in to this method (if not :data:`None`)
    - The value stored on the current blob
    - The value given by ``mimetypes.guess_type``
    - The default value ('application/octet-stream')
    
    .. note::
       The effect of uploading to an existing blob depends on the
       "versioning" and "lifecycle" policies defined on the blob's
       bucket.  In the absence of those policies, upl

In [50]:
blob.upload_from_filename('../data/retail_db/orders/part-00000') # upload file to blob

In [53]:
!gsutil ls -r gs://de_retail_db/pythondemo

gs://de_retail_db/pythondemo/:

gs://de_retail_db/pythondemo/retail_db/:

gs://de_retail_db/pythondemo/retail_db/orders/:
gs://de_retail_db/pythondemo/retail_db/orders/part-00000


In [52]:
help(gsclient.download_blob_to_file) #help on download blob to file

Help on method download_blob_to_file in module google.cloud.storage.client:

download_blob_to_file(blob_or_uri, file_obj, start=None, end=None, raw_download=False, if_etag_match=None, if_etag_not_match=None, if_generation_match=None, if_generation_not_match=None, if_metageneration_match=None, if_metageneration_not_match=None, timeout=60, checksum='auto', retry=<google.api_core.retry.retry_unary.Retry object at 0x10ea6b2e0>) method of google.cloud.storage.client.Client instance
    Download the contents of a blob object or blob URI into a file-like object.
    
    See https://cloud.google.com/storage/docs/downloading-objects
    
    Args:
        blob_or_uri (Union[             :class:`~google.cloud.storage.blob.Blob`,              str,             ]):
            The blob resource to pass or URI to download.
    
        file_obj (file):
            A file handle to which to write the blob's data.
    
        start (int):
            (Optional) The first byte in a range to be downlo

In [55]:
blob = bucket.blob('pythondemo/retail_db/orders/part-00000') #in memory
orders_csv = open('orders.csv', 'wb') #open file in binary mode
gsclient.download_blob_to_file(
    blob, #blob_or_uri
    orders_csv
)
orders_csv.close()

# Upload multiple files in GCS


In [58]:
!gsutil rm -r gs://de_retail_db/pythondemo/
!gsutil ls -r gs://de_retail_db/

Removing gs://de_retail_db/pythondemo/retail_db/orders/part-00000#1744030890762424...
/ [1 objects]                                                                   
Operation completed over 1 objects.                                              
gs://de_retail_db/retail_db/:
gs://de_retail_db/retail_db/create_db_tables_pg.sql
gs://de_retail_db/retail_db/load_db_tables_pg.sql
gs://de_retail_db/retail_db/schemas.json

gs://de_retail_db/retail_db/categories/:
gs://de_retail_db/retail_db/categories/part-00000

gs://de_retail_db/retail_db/customers/:
gs://de_retail_db/retail_db/customers/part-00000

gs://de_retail_db/retail_db/departments/:
gs://de_retail_db/retail_db/departments/part-00000

gs://de_retail_db/retail_db/order_items/:
gs://de_retail_db/retail_db/order_items/part-00000

gs://de_retail_db/retail_db/orders/:
gs://de_retail_db/retail_db/orders/part-00000

gs://de_retail_db/retail_db/products/:
gs://de_retail_db/retail_db/products/part-00000


In [59]:
import glob
import os 

In [ ]:
def get_file_names(base_dir):
    files = glob.glob(f'{base_dir}/**', recursive=True)
    return [file for file in files if os.path.isfile(file) and file.endswith('part-00000')] #same as below

In [ ]:
def get_file_names(base_dir):
    files = glob.glob(f'{base_dir}/**', recursive=True)
    return list(filter(lambda file: os.path.isfile(file) and file.endswith('part-00000'), files)) #same as above

In [70]:
base_dir = '../data/retail_db'
files = get_file_names(base_dir)
files

['../data/retail_db/customers/part-00000',
 '../data/retail_db/products/part-00000',
 '../data/retail_db/departments/part-00000',
 '../data/retail_db/order_items/part-00000',
 '../data/retail_db/orders/part-00000',
 '../data/retail_db/categories/part-00000']

In [71]:
file = files[0]
file

'../data/retail_db/customers/part-00000'

In [73]:
file.split('/')[2:] # split the file name and get the last 2 elements

['retail_db', 'customers', 'part-00000']

In [74]:
'/'.join(file.split('/')[2:]) # join the last 2 elements with '/' separator

'retail_db/customers/part-00000'

In [75]:
target_base_dir = 'pythondemo'

In [ ]:
from google.cloud import storage

gsclient = storage.Client()
bucket = gsclient.list_buckets()


In [ ]:
for file in files:
    file_name = '/'.join(file.split('/')[2:])
    file_name
    # blob = bucket.blob(f'{target_base_dir}/{file_name}')
    # blob.upload_from_filename(file) # upload files to blob



KeyboardInterrupt: 

In [ ]:
!gsutil ls -r gs://de_retail_db/pythondemo

In [ ]:
help(gsclient.list_blobs) #help on list blobs